<a href="https://colab.research.google.com/github/Dylan-Geraci/neuroimaging-tumor-detector/blob/main/notebooks/02_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Training

## Notebook Setup

Import Libraries

In [1]:
# --- Standard library ---
import os
import glob
import json
import pathlib

# --- Numerical / data ---
import numpy as np
import pandas as pd

# --- Imaging & plotting ---
from PIL import Image, UnidentifiedImageError
import matplotlib.pyplot as plt

# --- ML / data loading ---
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import datasets, transforms

# --- PyTorch training (add when you start training) ---
import torch
from torch import nn

# --- Utils ---
from tqdm.auto import tqdm  # nicer progress bars in Colab/Jupyter

# --- Colab ---
from google.colab import drive

Set Training Data Path

In [2]:
drive.mount('/content/drive')
TRAIN_PATH = "/content/drive/MyDrive/neuro-imaging/data/Training"

Mounted at /content/drive


In [3]:
os.makedirs("splits", exist_ok=True)

def write_list(indices, fname):
    with open(fname, "w") as f:
        for i in indices:
            f.write(str(pathlib.Path(base.samples[i][0]).resolve())+"\n")

write_list(train_idx, "splits/train.txt")
write_list(val_idx,   "splits/val.txt")
with open("class_to_idx.json","w") as f:
    json.dump(base.class_to_idx, f, indent=2)

NameError: name 'train_idx' is not defined